<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

In [1]:
# Parameters
run_dir = "/Users/daws/ADMF-PC/config/bollinger/results/20250627_093931"
config_name = "bollinger"
symbols = ["SPY"]
timeframe = "5m"
min_strategies_to_analyze = 20
sharpe_threshold = 1.0
correlation_threshold = 0.7
top_n_strategies = 10
ensemble_size = 5
calculate_all_performance = True
performance_limit = 100


# Trade & Risk Analysis Notebook\n\nThis notebook analyzes trading performance through orders, fills, and position events,\nwith special focus on risk management exits (stop loss, take profit, trailing stop).

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
import pandas as pd\nimport numpy as np\nimport json\nfrom pathlib import Path\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom datetime import datetime\n\n# Configure display\npd.set_option('display.max_columns', None)\npd.set_option('display.width', None)\npd.set_option('display.max_rows', 100)\n\n# Style\nplt.style.use('seaborn-v0_8-darkgrid')\nsns.set_palette('husl')

SyntaxError: unexpected character after line continuation character (3210297964.py, line 1)

## 1. Load Trace Data

In [ ]:
# Set the results directory\nresults_dir = Path('.')  # Assumes notebook is run from results directory\ntraces_dir = results_dir / 'traces'\n\n# Load all trace files\ndef load_trace_files(traces_dir):\n    """Load all trace files and return as dict of DataFrames."""\n    traces = {}\n    \n    # Strategy signals\n    signals_path = list(traces_dir.rglob('signals/*/*.parquet'))\n    if signals_path:\n        traces['signals'] = pd.read_parquet(signals_path[0])\n    \n    # Portfolio orders\n    orders_path = traces_dir / 'portfolio' / 'orders' / 'portfolio_orders.parquet'\n    if orders_path.exists():\n        traces['orders'] = pd.read_parquet(orders_path)\n    \n    # Execution fills\n    fills_path = traces_dir / 'execution' / 'fills' / 'execution_fills.parquet'\n    if fills_path.exists():\n        traces['fills'] = pd.read_parquet(fills_path)\n    \n    # Position events\n    pos_open_path = traces_dir / 'portfolio' / 'positions_open' / 'position_open.parquet'\n    if pos_open_path.exists():\n        traces['position_open'] = pd.read_parquet(pos_open_path)\n    \n    pos_close_path = traces_dir / 'portfolio' / 'positions_close' / 'position_close.parquet'\n    if pos_close_path.exists():\n        traces['position_close'] = pd.read_parquet(pos_close_path)\n    \n    return traces\n\ntraces = load_trace_files(traces_dir)\nprint(f"Loaded trace files: {list(traces.keys())}")

## 2. Parse Metadata and Build Trade Records

In [ ]:
def parse_metadata(df, col='metadata'):\n    """Parse JSON metadata column into separate columns."""\n    if col not in df.columns or len(df) == 0:\n        return df\n    \n    # Parse JSON metadata\n    metadata_list = []\n    for idx, row in df.iterrows():\n        try:\n            metadata = json.loads(row[col]) if row[col] else {}\n            metadata_list.append(metadata)\n        except:\n            metadata_list.append({})\n    \n    # Create DataFrame from metadata\n    metadata_df = pd.DataFrame(metadata_list)\n    \n    # Combine with original, avoiding duplicate columns\n    for col in metadata_df.columns:\n        if col not in df.columns:\n            df[col] = metadata_df[col]\n    \n    return df\n\n# Parse metadata for all traces\nfor key in ['orders', 'fills', 'position_open', 'position_close']:\n    if key in traces:\n        traces[key] = parse_metadata(traces[key])\n        print(f"Parsed {key}: {len(traces[key])} records")

## 3. Reconstruct Complete Trades

In [ ]:
def reconstruct_trades(traces):\n    """Reconstruct complete trades from orders, fills, and position events."""\n    trades = []\n    \n    # Match orders with fills\n    if 'orders' in traces and 'fills' in traces:\n        orders_df = traces['orders'].copy()\n        fills_df = traces['fills'].copy()\n        \n        # Add order index for matching\n        orders_df['order_idx'] = range(len(orders_df))\n        fills_df['fill_idx'] = range(len(fills_df))\n        \n        # Simple matching by bar index (assumes 1:1 order:fill)\n        for i, (_, order) in enumerate(orders_df.iterrows()):\n            if i < len(fills_df):\n                fill = fills_df.iloc[i]\n                \n                trade = {\n                    'order_idx': order['idx'],\n                    'fill_idx': fill['idx'],\n                    'symbol': order.get('symbol', 'UNKNOWN'),\n                    'side': order.get('side', 'UNKNOWN'),\n                    'quantity': order.get('quantity', 0),\n                    'order_price': order.get('price', 0),\n                    'fill_price': fill.get('price', 0),\n                    'order_time': order['ts'],\n                    'fill_time': fill['ts'],\n                    'strategy_id': order.get('strategy_id', 'unknown')\n                }\n                trades.append(trade)\n    \n    trades_df = pd.DataFrame(trades)\n    \n    # Add position events if available\n    if 'position_open' in traces and 'position_close' in traces:\n        # Match trades with position events\n        # This is simplified - in reality would need more sophisticated matching\n        pos_open = traces['position_open']\n        pos_close = traces['position_close']\n        \n        if len(pos_open) > 0 and len(pos_close) > 0:\n            # Add exit information to trades\n            for i, close_event in pos_close.iterrows():\n                exit_type = close_event.get('exit_type', 'unknown')\n                exit_reason = close_event.get('exit_reason', 'unknown')\n                if i < len(trades_df):\n                    trades_df.loc[trades_df.index[i], 'exit_type'] = exit_type\n                    trades_df.loc[trades_df.index[i], 'exit_reason'] = exit_reason\n    \n    return trades_df\n\ntrades_df = reconstruct_trades(traces)\nprint(f"Reconstructed {len(trades_df)} trades")

## 4. Analyze Risk Management Exits

In [ ]:
# Analyze position close events for risk exits\nif 'position_close' in traces:\n    pos_close = traces['position_close']\n    \n    print("=== Risk Management Exit Analysis ===")\n    print(f"Total positions closed: {len(pos_close)}")\n    \n    # Count exit types\n    if 'exit_type' in pos_close.columns:\n        exit_counts = pos_close['exit_type'].value_counts()\n        print("\nExit types:")\n        for exit_type, count in exit_counts.items():\n            print(f"  {exit_type}: {count} ({count/len(pos_close)*100:.1f}%)")\n    \n    # Analyze exit reasons\n    if 'exit_reason' in pos_close.columns:\n        print("\nExit reasons:")\n        for i, row in pos_close.iterrows():\n            print(f"  Trade {i+1}: {row.get('exit_type', 'unknown')} - {row.get('exit_reason', 'unknown')}")\n            if hasattr(row, 'realized_pnl'):\n                print(f"    PnL: ${row['realized_pnl']:.2f}")

## 5. Signal vs Position Analysis

In [ ]:
# Analyze signal persistence after risk exits\nif 'signals' in traces and 'position_close' in traces:\n    signals_df = traces['signals']\n    pos_close = traces['position_close']\n    \n    print("=== Signal Persistence After Risk Exits ===")\n    \n    # For each position close due to risk\n    risk_closes = pos_close[pos_close.get('exit_type', '').isin(['stop_loss', 'trailing_stop'])] if 'exit_type' in pos_close.columns else pd.DataFrame()\n    \n    if len(risk_closes) > 0:\n        for _, close_event in risk_closes.iterrows():\n            close_bar = close_event['idx']\n            \n            # Check signal value at close and next few bars\n            next_signals = signals_df[signals_df['idx'].between(close_bar, close_bar + 10)]\n            \n            if len(next_signals) > 0:\n                print(f"\nRisk exit at bar {close_bar}:")\n                print(f"  Exit type: {close_event.get('exit_type', 'unknown')}")\n                print(f"  Signal values after exit:")\n                for _, sig in next_signals.iterrows():\n                    print(f"    Bar {sig['idx']}: signal = {sig['val']}")

## 6. Performance Metrics by Exit Type

In [ ]:
# Calculate performance metrics grouped by exit type\nif 'position_close' in traces and 'realized_pnl' in traces['position_close'].columns:\n    pos_close = traces['position_close']\n    \n    # Group by exit type\n    if 'exit_type' in pos_close.columns:\n        metrics_by_exit = pos_close.groupby('exit_type').agg({\n            'realized_pnl': ['count', 'sum', 'mean', 'std'],\n            'idx': ['min', 'max']  # First and last bar\n        }).round(2)\n        \n        print("=== Performance by Exit Type ===")\n        print(metrics_by_exit)\n        \n        # Visualize\n        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))\n        \n        # Count by exit type\n        exit_counts = pos_close['exit_type'].value_counts()\n        exit_counts.plot(kind='bar', ax=ax1, title='Number of Exits by Type')\n        ax1.set_xlabel('Exit Type')\n        ax1.set_ylabel('Count')\n        \n        # PnL by exit type\n        pos_close.boxplot(column='realized_pnl', by='exit_type', ax=ax2)\n        ax2.set_title('PnL Distribution by Exit Type')\n        ax2.set_xlabel('Exit Type')\n        ax2.set_ylabel('Realized PnL ($)')\n        \n        plt.tight_layout()\n        plt.show()

## 7. Trade Duration Analysis

In [ ]:
# Analyze trade durations\nif 'position_open' in traces and 'position_close' in traces:\n    pos_open = traces['position_open']\n    pos_close = traces['position_close']\n    \n    if len(pos_open) > 0 and len(pos_close) > 0:\n        # Calculate trade durations (simplified - assumes matching order)\n        durations = []\n        for i in range(min(len(pos_open), len(pos_close))):\n            open_bar = pos_open.iloc[i]['idx']\n            close_bar = pos_close.iloc[i]['idx']\n            duration = close_bar - open_bar\n            exit_type = pos_close.iloc[i].get('exit_type', 'unknown')\n            \n            durations.append({\n                'duration_bars': duration,\n                'exit_type': exit_type,\n                'pnl': pos_close.iloc[i].get('realized_pnl', 0)\n            })\n        \n        duration_df = pd.DataFrame(durations)\n        \n        print("=== Trade Duration Analysis ===")\n        print(f"Average duration: {duration_df['duration_bars'].mean():.1f} bars")\n        print(f"Median duration: {duration_df['duration_bars'].median():.1f} bars")\n        \n        # Duration by exit type\n        if 'exit_type' in duration_df.columns:\n            print("\nAverage duration by exit type:")\n            for exit_type, group in duration_df.groupby('exit_type'):\n                print(f"  {exit_type}: {group['duration_bars'].mean():.1f} bars")

## 8. Order Flow Analysis

In [ ]:
# Analyze order patterns\nif 'orders' in traces:\n    orders_df = traces['orders']\n    \n    print("=== Order Flow Analysis ===")\n    print(f"Total orders: {len(orders_df)}")\n    \n    # Order frequency\n    if len(orders_df) > 1:\n        order_gaps = orders_df['idx'].diff().dropna()\n        print(f"\nAverage bars between orders: {order_gaps.mean():.1f}")\n        print(f"Median bars between orders: {order_gaps.median():.1f}")\n        \n        # Check for immediate re-entry after exit\n        immediate_reentries = (order_gaps <= 1).sum()\n        print(f"\nImmediate re-entries (≤1 bar): {immediate_reentries}")\n        \n        # Visualize order frequency\n        plt.figure(figsize=(10, 5))\n        plt.hist(order_gaps, bins=50, edgecolor='black', alpha=0.7)\n        plt.xlabel('Bars Between Orders')\n        plt.ylabel('Frequency')\n        plt.title('Distribution of Time Between Orders')\n        plt.axvline(order_gaps.mean(), color='red', linestyle='--', label=f'Mean: {order_gaps.mean():.1f}')\n        plt.axvline(order_gaps.median(), color='green', linestyle='--', label=f'Median: {order_gaps.median():.1f}')\n        plt.legend()\n        plt.show()

## 9. Risk Management Effectiveness

In [ ]:
# Analyze effectiveness of risk management\nif 'position_close' in traces and 'realized_pnl' in traces['position_close'].columns:\n    pos_close = traces['position_close']\n    \n    print("=== Risk Management Effectiveness ===")\n    \n    # Calculate metrics\n    total_trades = len(pos_close)\n    profitable_trades = (pos_close['realized_pnl'] > 0).sum() if 'realized_pnl' in pos_close.columns else 0\n    \n    if total_trades > 0:\n        win_rate = profitable_trades / total_trades * 100\n        avg_win = pos_close[pos_close['realized_pnl'] > 0]['realized_pnl'].mean() if profitable_trades > 0 else 0\n        avg_loss = pos_close[pos_close['realized_pnl'] < 0]['realized_pnl'].mean() if (total_trades - profitable_trades) > 0 else 0\n        \n        print(f"Total trades: {total_trades}")\n        print(f"Win rate: {win_rate:.1f}%")\n        print(f"Average win: ${avg_win:.2f}")\n        print(f"Average loss: ${avg_loss:.2f}")\n        \n        if avg_loss != 0:\n            profit_factor = abs(avg_win / avg_loss)\n            print(f"Profit factor: {profit_factor:.2f}")\n        \n        # Analyze by exit type\n        if 'exit_type' in pos_close.columns:\n            print("\nWin rate by exit type:")\n            for exit_type, group in pos_close.groupby('exit_type'):\n                wins = (group['realized_pnl'] > 0).sum()\n                total = len(group)\n                win_pct = wins / total * 100 if total > 0 else 0\n                print(f"  {exit_type}: {win_pct:.1f}% ({wins}/{total})")

## 10. Summary & Recommendations

In [ ]:
print("=== SUMMARY ===")\nprint(f"\nData Quality:")\nfor key, df in traces.items():\n    print(f"  {key}: {len(df)} records")\n\nprint(f"\nKey Findings:")\nif 'orders' in traces and 'fills' in traces:\n    print(f"  - Order fill rate: {len(traces['fills'])/len(traces['orders'])*100:.1f}%")\n\nif 'position_close' in traces and 'exit_type' in traces['position_close'].columns:\n    risk_exits = traces['position_close']['exit_type'].isin(['stop_loss', 'take_profit', 'trailing_stop']).sum()\n    print(f"  - Risk management exits: {risk_exits} ({risk_exits/len(traces['position_close'])*100:.1f}%)")\n\nprint("\n=== RECOMMENDATIONS ===")\nprint("1. Check for immediate re-entry after risk exits")\nprint("2. Analyze signal persistence after stop-loss exits")\nprint("3. Consider implementing a 'cooldown' period after risk exits")\nprint("4. Review risk parameters if too many stop-loss exits")